<img src="../../images/qiskit-heading.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="500 px" align="left">

# Qiskit Aer: Noise Transformation

The latest version of this notebook is available on https://github.com/Qiskit/qiskit-tutorials.

## Introduction

This notebook shows how to use the Qiskit Aer `noise.utils.approximate_quantum_error` and `noise.utils. approximate_noise_model` to transform quantum noise channels into a different, more suitable, noise channel.

Our guiding example is Clifford simulation. A Clifford simulator can efficiently simulate quantum computations which include gates only from a limited, non-universal set of gates (the Clifford gates). Not all quantum noises can be added to such simulations; hence, we aim to find a "close" noise channel which can be simulated in a Clifford simulator.

We begin by importing the transofrmation functions.

In [ ]:
from qiskit.providers.aer.noise.utils import approximate_quantum_error
from qiskit.providers.aer.noise.utils import approximate_noise_model

The name "approximate" suggests that this functions generate the closest (in the Hilbert-Schmidt metric) error possible to the given one.

We demonstrate the approximation using several standard error channels defined in Qiskit.

In [ ]:
from qiskit.providers.aer.noise.errors.standard_errors import amplitude_damping_error
from qiskit.providers.aer.noise.errors.standard_errors import reset_error
from qiskit.providers.aer.noise.errors.standard_errors import pauli_error

## Example: Approximating amplitude damping noise with reset noise.

**Amplitude damping** noise is described by a single parameter $0\le \gamma \le 1$ and given by the Kraus operators:

$$\left(\begin{array}{cc}
1 & 0\\
0 & \sqrt{1-\gamma}
\end{array}\right),\left(\begin{array}{cc}
0 & \gamma\\
0 & 0
\end{array}\right)$$

**Reset** error is described by probabilities $0\le p, q\le 1$ such that $p+q\le 1$ and given by the Kraus operators:

$$\left(\begin{array}{cc}
\sqrt{p} & 0\\
0 & 0
\end{array}\right),\left(\begin{array}{cc}
0 & \sqrt{p}\\
0 & 0
\end{array}\right),\left(\begin{array}{cc}
0 & 0\\
\sqrt{q} & 0
\end{array}\right),\left(\begin{array}{cc}
0 & 0\\
0 & \sqrt{q}
\end{array}\right)$$

